In [1]:
import os
import re
import time
import datetime
import requests
import pandas as pd
import requests as rq
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from IPython.display import clear_output
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
def foreigner(html):
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('p')
    # print(links)
    # <span itemprop="birthPlace">

    links = soup.find_all(['span'] , attrs = { 'itemprop' : 'birthPlace'})
    for link in links:
        for c in (link.descendants):
            aa = ((c.find_next('a')).attrs)
            if 'country=US' in aa['href']:
                return False
    return True

## Should exccute the chromedriver and get the page source

In [3]:
def league(html):
    result = []
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('div', attrs = {'class': 'leaderboard_wrapper' , 'id': 'all_leaderboard'})

    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        a = comment.find('div_leaderboard')
        if (a != -1):
            #print(a,comment)
            break
            
    ts = BeautifulSoup(comment.string, 'html.parser')
    ts = ts.find_all('div', {'id' : 'leaderboard_all_league' , 'class' : 'data_grid_box'})
    for tt in ts:
        kk = tt.table.find_all('a')
        for k in kk:
            result.append((k.text))
    return result

## Should exceute the chromedriver and get the page source

In [4]:
def allstar(html):
    result = []
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('div', attrs = {'class': 'leaderboard_wrapper' , 'id': 'all_leaderboard'})

    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        a = comment.find('div_leaderboard')
        if (a != -1):
            #print(a,comment)
            break
    ts = BeautifulSoup(comment.string, 'html.parser')
    ts = ts.find_all('div', {'id' : 'leaderboard_allstar' , 'class' : 'data_grid_box'})
    for tt in ts:
        kk = tt.table.find_all('a')
        for k in kk:
            result.append((k.text))
            
    result = list(map (lambda x : str(int(x.split()[0])-1 ) + '-' + (x.split()[0][2:])  , result))
    return result

In [5]:
def Hall(html):
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('li' , {'class'  : 'bling_special bling_hof'})
    for link in links:
        if link.text == "Hall of Fame":
            return True

In [6]:
def btn_click(wd,url):
    wd.get(url)
    script = "vjs_addClass(document.getElementById('leaderboard_allstar'),'show_all')"
    script2 = "vjs_addClass(document.getElementById('leaderboard_allleague'),'show_all')"
    btn = wd.execute_script(script)
    btn = wd.execute_script(script2)
    return(wd.page_source)

In [7]:
def get_all_info(wd,url):
    html = btn_click(wd,url)
    result = ([Hall(html) ,league(html), allstar(html), foreigner(html)])
    return (result)

In [8]:
all_data = []
df = pd.read_csv('./info/player_my_df.csv')
info = pd.read_excel('./info/info.xlsx')

dirs = !ls ./data
dirs_no = sorted(list(map (lambda x : int(x.split('.')[0]),dirs)))

## Crawl the table

In [11]:
def per_game_table(html):
    
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')    
    links = soup.find_all('table', { 'id' : 'per_game'} )
    
    if (len(links) == 0):
        return

    rows = links[0].find_all('tr')

    colname = []
    rowname = []
    for header in links[0].find_all('th', {'scope' : 'col'}):
        colname.append(header.text)
        
    for header in links[0].find_all('th', {'scope' : 'row'}):
        rowname.append(header.text)
        
    table =[]
    df = pd.DataFrame()
    
    rnum = 0
    for row in links[0].find_all('tr'):
        table = []
        columns = row.find_all('td')
        
        for column in columns:
            ex = column.text.split('-')
            try:
                if len(ex) > 1 and int(ex[0]):
                    rowname.insert(rnum, column.text)
                    continue
            except:
                pass
            table.append(column.text)
            
        if not table :
            continue   
            
        rnum = rnum + 1    
        df = df.append(pd.Series(table),ignore_index=True)
    
    df[colname[0]] = pd.Series(rowname)
    df.columns = colname[-len(colname) + 1 :] + colname[:-len(colname) + 1 ]
    return(df[colname])

## Load the all data processed

In [ ]:
import pickle
dirs = !ls ./data
dirs_no = sorted(list(map (lambda x : int(x.split('.')[0]),dirs)))

with open ('./info/all_data', 'rb') as fp:
    data = pickle.load(fp)
      
info = pd.read_excel('./info/info2.xlsx')

col = ['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

url = pd.read_csv('./info/player_my_df.csv')['5']
df = pd.DataFrame(columns=col)   
row = len(data)
ind = info.columns

for i in range(row):
    if(len(dirs_no) > 0):
        if i != dirs_no[0]:
            continue
        dirs_no.pop(0)
    
    response = rq.get(url[i])
    html = response.text
    per_game_table(html) 
    my_list = per_game_table(html)
    
    
    pp = list(my_list['Season']).index('Career')
    my_list = my_list.loc[:pp,:]

    r , c = ((my_list.shape))
    
    small_df = my_list


    for j in range(len(col)):
        if col[j] not in small_df.columns:
            small_df[col[j]] = pd.Series([' ']* r)
    
    
    ## Process fix award ##
    ## [None, ['1988-89'], ['1989-90'], False]
    award = data[i]
    inf = list(info.iloc[i,:])
 
    if(award[0]):
        small_df['名人堂']= pd.Series([1]*r)
    else:
        small_df['名人堂']= pd.Series([0]*r)

    if(award[3]):
        small_df['外籍球員']= pd.Series([1]*r)
    else:
        small_df['外籍球員']= pd.Series([0]*r)

    small_df['是否入圍全明星'] = pd.Series([0]*r)
    small_df['當年度獎項次數'] = pd.Series([0]*r)
    small_df['當年度獎項YN'] = pd.Series([0]*r)
    
    
    #([Hall(html) ,league(html), allstar(html), foreigner(html)])
    
    all_series = list(small_df.iloc[:,0])
    
    for m in range(len(award[2])):
        for n in range(len(all_series)):
            if(award[2][m] == all_series[n]):
                small_df['是否入圍全明星'][n] = 1
                
    small_df['當年度獎項次數'][r-1] = sum(list(small_df['當年度獎項次數']))
    
    for m in range(len(award[1])):
        for n in range(len(all_series)):
            if(award[1][m] == all_series[n]):
                small_df['當年度獎項次數'][n] += 1
                small_df['當年度獎項YN'][n] = 1

    ## Process info ##
    for k in range(len(inf)):
        small_df[ind[k]]= pd.Series([inf[k]] * r)
   
    year_play = pd.Series([0]*r)  
    all_series.pop(-1)    
    compact_id = list(set(all_series))
    compact_id = sorted(compact_id,key= lambda x : int(x.split('-')[0]))
   
    for j in range(len(all_series)):
        front = list(small_df['Tm'])[:j]
        count = 0
        for k in range(len(front)):
            if 'Did' in front[k]:
                count += 1
        year_play[j] = compact_id.index(all_series[j]) - count + 1 
        
    year_play[j+1] = list(year_play)[-2]
    
    small_df['年資'] = year_play
    
    df = pd.concat([small_df,df],axis=0,join='outer',ignore_index=True,sort=False)
    
    if i % 20 == 1:
        print(i)

    if i % 200 == 1:
        clear_output()
       

61
81
101


In [40]:
cols = df.columns.tolist()
cols = cols[-15:] + cols[:-15]
cols = ['FirstName', 'LastName', 'From', 'To',\
        '位置', '呎', '吋', '磅', 'Colleges','出生年', '出生月', \
        '出生日', '選秀順位','名人堂' , '當年度獎項次數', '當年度獎項YN' , '是否入圍全明星','外籍球員', '年資', \
        'Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP',\
        'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',\
        'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

In [39]:
ss = df[cols]
ss.to_excel('Final7.xlsx')